# Coursework 1 - Question 2

## Web References

- [k-means document clustering using Apache Mahout command line](https://datasciencetutos.wordpress.com/2016/08/04/k-means-document-clustering-using-apache-mahout-command-line/)
- [https://bickson.blogspot.com/2011/09/understanding-mahout-k-means-clustering.html](https://bickson.blogspot.com/2011/09/understanding-mahout-k-means-clustering.html)
- [Chapter 7. Introduction to clustering – Mahout in Action](http://devguis.com/chapter-7-introduction-to-clustering-mahout-in-action.html)
- [Package org.apache.mahout.common.distance](https://mahout.apache.org/docs/0.13.0/api/docs/mahout-mr/org/apache/mahout/common/distance/package-summary.html)

## Data Preperation

### Upload Data HDFS

In [1]:
%%bash
# change to the coursework directory
cd ~/code/dsm010-2021-oct/coursework_01/

# copy the input documents
 hadoop fs -copyFromLocal data/raw/western_classics/british-fiction-corpus dsm010/british-fiction-corpus

# verify the file uploads
hadoop fs -ls dsm010/british-fiction-corpus

Found 27 items
-rw-r--r--   3 jfoul001 users     378987 2022-01-01 13:03 dsm010/british-fiction-corpus/ABronte_Agnes.txt
-rw-r--r--   3 jfoul001 users     930593 2022-01-01 13:03 dsm010/british-fiction-corpus/ABronte_Tenant.txt
-rw-r--r--   3 jfoul001 users     894304 2022-01-01 13:03 dsm010/british-fiction-corpus/Austen_Emma.txt
-rw-r--r--   3 jfoul001 users     683758 2022-01-01 13:03 dsm010/british-fiction-corpus/Austen_Pride.txt
-rw-r--r--   3 jfoul001 users     678691 2022-01-01 13:03 dsm010/british-fiction-corpus/Austen_Sense.txt
-rw-r--r--   3 jfoul001 users    1026320 2022-01-01 13:03 dsm010/british-fiction-corpus/CBronte_Jane.txt
-rw-r--r--   3 jfoul001 users     506144 2022-01-01 13:03 dsm010/british-fiction-corpus/CBronte_Professor.txt
-rw-r--r--   3 jfoul001 users    1104704 2022-01-01 13:03 dsm010/british-fiction-corpus/CBronte_Villette.txt
-rw-r--r--   3 jfoul001 users    1964819 2022-01-01 13:03 dsm010/british-fiction-corpus/Dickens_Bleak.txt
-rw-r--r--   3 jfoul001 user

### Convert the dataset to SequenceFiles

SequenceFiles are flat files consisting of binary key/value pairs. Each document is represented as a key-value pair. there the key is the document id and value is its content.

In [2]:
%%bash
mahout seqdirectory \
    -i dsm010/british-fiction-corpus \
    -o dsm010/british-fiction-corpus-seqfiles \
    -c UTF-8 \
    -chunk 5

MAHOUT_LOCAL is not set; adding HADOOP_CONF_DIR to classpath.
Running on hadoop, using /opt/hadoop/current/bin/hadoop and HADOOP_CONF_DIR=/opt/hadoop/current/etc/hadoop
MAHOUT-JOB: /opt/mahout/current/mahout-examples-0.13.0-job.jar


22/01/01 13:03:35 INFO AbstractJob: Command line arguments: {--charset=[UTF-8], --chunkSize=[5], --endPhase=[2147483647], --fileFilterClass=[org.apache.mahout.text.PrefixAdditionFilter], --input=[dsm010/british-fiction-corpus], --keyPrefix=[], --method=[mapreduce], --output=[dsm010/british-fiction-corpus-seqfiles], --startPhase=[0], --tempDir=[temp]}
22/01/01 13:03:36 INFO deprecation: mapred.input.dir is deprecated. Instead, use mapreduce.input.fileinputformat.inputdir
22/01/01 13:03:36 INFO deprecation: mapred.compress.map.output is deprecated. Instead, use mapreduce.map.output.compress
22/01/01 13:03:36 INFO deprecation: mapred.output.dir is deprecated. Instead, use mapreduce.output.fileoutputformat.outputdir
22/01/01 13:03:36 INFO DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at lena-master/128.86.245.64:8032
22/01/01 13:03:36 INFO JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/jfoul001/.staging/job_1626049283275_239015
22/01/01

In [3]:
%%bash
hadoop fs -ls dsm010/british-fiction-corpus-seqfiles

Found 7 items
-rw-r--r--   3 jfoul001 users          0 2022-01-01 13:03 dsm010/british-fiction-corpus-seqfiles/_SUCCESS
-rw-r--r--   3 jfoul001 users    2679688 2022-01-01 13:03 dsm010/british-fiction-corpus-seqfiles/part-m-00000
-rw-r--r--   3 jfoul001 users    2490469 2022-01-01 13:03 dsm010/british-fiction-corpus-seqfiles/part-m-00001
-rw-r--r--   3 jfoul001 users    2295247 2022-01-01 13:03 dsm010/british-fiction-corpus-seqfiles/part-m-00002
-rw-r--r--   3 jfoul001 users    2108742 2022-01-01 13:03 dsm010/british-fiction-corpus-seqfiles/part-m-00003
-rw-r--r--   3 jfoul001 users    2065709 2022-01-01 13:03 dsm010/british-fiction-corpus-seqfiles/part-m-00004
-rw-r--r--   3 jfoul001 users    1887752 2022-01-01 13:03 dsm010/british-fiction-corpus-seqfiles/part-m-00005


### Convert sequenceFiles to sparse vector files

In [4]:
%%bash
mahout seq2sparse \
    -nv \
    -i dsm010/british-fiction-corpus-seqfiles \
    -o dsm010/british-fiction-corpus-vectors \

MAHOUT_LOCAL is not set; adding HADOOP_CONF_DIR to classpath.
Running on hadoop, using /opt/hadoop/current/bin/hadoop and HADOOP_CONF_DIR=/opt/hadoop/current/etc/hadoop
MAHOUT-JOB: /opt/mahout/current/mahout-examples-0.13.0-job.jar


22/01/01 13:03:57 INFO SparseVectorsFromSequenceFiles: Maximum n-gram size is: 1
22/01/01 13:03:57 INFO SparseVectorsFromSequenceFiles: Minimum LLR value: 1.0
22/01/01 13:03:57 INFO SparseVectorsFromSequenceFiles: Number of reduce tasks: 1
22/01/01 13:03:57 INFO SparseVectorsFromSequenceFiles: Tokenizing documents in dsm010/british-fiction-corpus-seqfiles
22/01/01 13:03:57 INFO DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at lena-master/128.86.245.64:8032
22/01/01 13:03:58 INFO JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/jfoul001/.staging/job_1626049283275_239017
22/01/01 13:03:58 INFO FileInputFormat: Total input files to process : 6
22/01/01 13:03:58 INFO JobSubmitter: number of splits:6
22/01/01 13:03:58 INFO JobSubmitter: Submitting tokens for job: job_1626049283275_239017
22/01/01 13:03:58 INFO JobSubmitter: Executing with tokens: []
22/01/01 13:03:58 INFO Configuration: resource-types.xml not found
22/01/01 13:03:58 INFO R

In [5]:
%%bash
hadoop fs -ls dsm010/british-fiction-corpus-vectors

Found 7 items
drwxr-xr-x   - jfoul001 users          0 2022-01-01 13:05 dsm010/british-fiction-corpus-vectors/df-count
-rw-r--r--   3 jfoul001 users     800064 2022-01-01 13:04 dsm010/british-fiction-corpus-vectors/dictionary.file-0
-rw-r--r--   3 jfoul001 users     775353 2022-01-01 13:05 dsm010/british-fiction-corpus-vectors/frequency.file-0
drwxr-xr-x   - jfoul001 users          0 2022-01-01 13:05 dsm010/british-fiction-corpus-vectors/tf-vectors
drwxr-xr-x   - jfoul001 users          0 2022-01-01 13:06 dsm010/british-fiction-corpus-vectors/tfidf-vectors
drwxr-xr-x   - jfoul001 users          0 2022-01-01 13:04 dsm010/british-fiction-corpus-vectors/tokenized-documents
drwxr-xr-x   - jfoul001 users          0 2022-01-01 13:04 dsm010/british-fiction-corpus-vectors/wordcount


In [6]:
%%bash
hadoop fs -ls dsm010/british-fiction-corpus-vectors/tf-vectors

Found 2 items
-rw-r--r--   3 jfoul001 users          0 2022-01-01 13:05 dsm010/british-fiction-corpus-vectors/tf-vectors/_SUCCESS
-rw-r--r--   3 jfoul001 users    2680027 2022-01-01 13:05 dsm010/british-fiction-corpus-vectors/tf-vectors/part-r-00000


## Clustering

### Canopy Clustering

To have initial centroids values for the k-means algorithm, run canopy clustering on the TF-IDF vectors.

In [7]:
%%bash
mahout canopy \
    -i dsm010/british-fiction-corpus-vectors/tf-vectors \
    -ow \
    -o dsm010/british-fiction-corpus-canopy-centroids \
    -dm org.apache.mahout.common.distance.CosineDistanceMeasure \
    -t1 0.5 \
    -t2 0.3

MAHOUT_LOCAL is not set; adding HADOOP_CONF_DIR to classpath.
Running on hadoop, using /opt/hadoop/current/bin/hadoop and HADOOP_CONF_DIR=/opt/hadoop/current/etc/hadoop
MAHOUT-JOB: /opt/mahout/current/mahout-examples-0.13.0-job.jar


22/01/01 13:06:44 INFO AbstractJob: Command line arguments: {--distanceMeasure=[org.apache.mahout.common.distance.CosineDistanceMeasure], --endPhase=[2147483647], --input=[dsm010/british-fiction-corpus-vectors/tf-vectors], --method=[mapreduce], --output=[dsm010/british-fiction-corpus-canopy-centroids], --overwrite=null, --startPhase=[0], --t1=[0.5], --t2=[0.3], --tempDir=[temp]}
22/01/01 13:06:44 INFO CanopyDriver: Build Clusters Input: dsm010/british-fiction-corpus-vectors/tf-vectors Out: dsm010/british-fiction-corpus-canopy-centroids Measure: org.apache.mahout.common.distance.CosineDistanceMeasure@4b7c1779 t1: 0.5 t2: 0.3
22/01/01 13:06:44 INFO DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at lena-master/128.86.245.64:8032
22/01/01 13:06:45 INFO JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/jfoul001/.staging/job_1626049283275_239035
22/01/01 13:06:45 INFO FileInputFormat: Total input files to process : 1
22/01/01 13:06:45 INFO Jo

In [8]:
%%bash
hadoop fs -ls dsm010/british-fiction-corpus-canopy-centroids/

Found 1 items
drwxr-xr-x   - jfoul001 users          0 2022-01-01 13:07 dsm010/british-fiction-corpus-canopy-centroids/clusters-0-final


### Perform K-Means Clustering

In [9]:
%%bash
mahout kmeans \
    -i dsm010/british-fiction-corpus-vectors/tf-vectors \
    -c dsm010/british-fiction-corpus-canopy-centroids \
    -o hdfs://lena/user/jfoul001/dsm010/british-fiction-corpus-kmeans-clusters \
    -dm org.apache.mahout.common.distance.CosineDistanceMeasure \
    -cl -cd 0.1 -ow -x 20 \
    -k 10

MAHOUT_LOCAL is not set; adding HADOOP_CONF_DIR to classpath.
Running on hadoop, using /opt/hadoop/current/bin/hadoop and HADOOP_CONF_DIR=/opt/hadoop/current/etc/hadoop
MAHOUT-JOB: /opt/mahout/current/mahout-examples-0.13.0-job.jar


22/01/01 13:07:10 INFO AbstractJob: Command line arguments: {--clustering=null, --clusters=[dsm010/british-fiction-corpus-canopy-centroids], --convergenceDelta=[0.1], --distanceMeasure=[org.apache.mahout.common.distance.CosineDistanceMeasure], --endPhase=[2147483647], --input=[dsm010/british-fiction-corpus-vectors/tf-vectors], --maxIter=[20], --method=[mapreduce], --numClusters=[10], --output=[hdfs://lena/user/jfoul001/dsm010/british-fiction-corpus-kmeans-clusters], --overwrite=null, --startPhase=[0], --tempDir=[temp]}
22/01/01 13:07:11 INFO HadoopUtil: Deleting dsm010/british-fiction-corpus-canopy-centroids
22/01/01 13:07:11 INFO ZlibFactory: Successfully loaded & initialized native-zlib library
22/01/01 13:07:11 INFO CodecPool: Got brand-new compressor [.deflate]
22/01/01 13:07:11 INFO RandomSeedGenerator: Wrote 10 Klusters to dsm010/british-fiction-corpus-canopy-centroids/part-randomSeed
22/01/01 13:07:11 INFO KMeansDriver: Input: dsm010/british-fiction-corpus-vectors/tf-vectors Clu

In [10]:
%%bash
hadoop fs -ls dsm010/british-fiction-corpus-kmeans-clusters

Found 5 items
-rw-r--r--   3 jfoul001 users        194 2022-01-01 13:07 dsm010/british-fiction-corpus-kmeans-clusters/_policy
drwxr-xr-x   - jfoul001 users          0 2022-01-01 13:07 dsm010/british-fiction-corpus-kmeans-clusters/clusteredPoints
drwxr-xr-x   - jfoul001 users          0 2022-01-01 13:07 dsm010/british-fiction-corpus-kmeans-clusters/clusters-0
drwxr-xr-x   - jfoul001 users          0 2022-01-01 13:07 dsm010/british-fiction-corpus-kmeans-clusters/clusters-1
drwxr-xr-x   - jfoul001 users          0 2022-01-01 13:07 dsm010/british-fiction-corpus-kmeans-clusters/clusters-2-final


In [11]:
%%bash
hadoop fs -ls dsm010/british-fiction-corpus-kmeans-clusters/clusteredPoints

Found 2 items
-rw-r--r--   3 jfoul001 users          0 2022-01-01 13:07 dsm010/british-fiction-corpus-kmeans-clusters/clusteredPoints/_SUCCESS
-rw-r--r--   3 jfoul001 users    2680742 2022-01-01 13:07 dsm010/british-fiction-corpus-kmeans-clusters/clusteredPoints/part-m-00000


In [12]:
%%bash
hadoop fs -ls dsm010/british-fiction-corpus-kmeans-clusters/clusters-2-final

Found 3 items
-rw-r--r--   3 jfoul001 users          0 2022-01-01 13:07 dsm010/british-fiction-corpus-kmeans-clusters/clusters-2-final/_SUCCESS
-rw-r--r--   3 jfoul001 users        194 2022-01-01 13:07 dsm010/british-fiction-corpus-kmeans-clusters/clusters-2-final/_policy
-rw-r--r--   3 jfoul001 users    2489304 2022-01-01 13:07 dsm010/british-fiction-corpus-kmeans-clusters/clusters-2-final/part-r-00000


## Interpreting Clustering Final Results

### Output the cluster results

In [13]:
%%bash
mahout clusterdump \
    -dt sequencefile \
    -d dsm010/british-fiction-corpus-vectors/dictionary.file-* \
    -i dsm010/british-fiction-corpus-kmeans-clusters/clusters-2-final  \
    -o ~/code/dsm010-2021-oct/coursework_01/data/output/british-fiction-corpus-clusters.txt -b 100 \
    -p dsm010/british-fiction-corpus-kmeans-clusters/clusteredPoints \
    -n 20 --evaluate

MAHOUT_LOCAL is not set; adding HADOOP_CONF_DIR to classpath.
Running on hadoop, using /opt/hadoop/current/bin/hadoop and HADOOP_CONF_DIR=/opt/hadoop/current/etc/hadoop
MAHOUT-JOB: /opt/mahout/current/mahout-examples-0.13.0-job.jar


22/01/01 13:08:09 INFO AbstractJob: Command line arguments: {--dictionary=[dsm010/british-fiction-corpus-vectors/dictionary.file-*], --dictionaryType=[sequencefile], --distanceMeasure=[org.apache.mahout.common.distance.SquaredEuclideanDistanceMeasure], --endPhase=[2147483647], --evaluate=null, --input=[dsm010/british-fiction-corpus-kmeans-clusters/clusters-2-final], --numWords=[20], --output=[/home/jfoul001/code/dsm010-2021-oct/coursework_01/data/output/british-fiction-corpus-clusters.txt], --outputFormat=[TEXT], --pointsDir=[dsm010/british-fiction-corpus-kmeans-clusters/clusteredPoints], --startPhase=[0], --substring=[100], --tempDir=[temp]}
22/01/01 13:08:10 INFO AbstractJob: Command line arguments: {--clusteredPoints=[dsm010/british-fiction-corpus-kmeans-clusters/clusteredPoints], --distanceMeasure=[org.apache.mahout.common.distance.SquaredEuclideanDistanceMeasure], --endPhase=[2147483647], --input=[dsm010/british-fiction-corpus-kmeans-clusters/clusters-2-final], --maxIter=[5], --me

### Review Cluster Dump Output

The final lines give a statistical evaluation of how good the clustering solution was. You can disregard the CDbw metries. Low inter-cluster density and high intra-cluster density indicates a good solution. The beginning of the file gives representative members (terms) in each cluster, as well as the actual documents in the cluster.

In [14]:
%%bash
tail -n 5 ~/code/dsm010-2021-oct/coursework_01/data/output/british-fiction-corpus-clusters.txt

Inter-Cluster Density: 0.47373238523434597
Intra-Cluster Density: 0.5515443694626683
CDbw Inter-Cluster Density: 0.0
CDbw Intra-Cluster Density: 0.8801331687549373
CDbw Separation: 2.530054241000007E8


##  Hyperparameter Optimization

In [34]:
%%bash
#!/bin/bash

# range to use for k
k_start=3
k_end=3

# the path to the vectors and dictionary
path_vectors=dsm010/british-fiction-corpus-vectors/tf-vectors
path_dictionary=dsm010/british-fiction-corpus-vectors/dictionary.file-*

# the output base path for the clusters and the result local output path
path_hdfs_base=hdfs://lena/user/jfoul001/
path_clusters_base=dsm010/british-fiction-corpus-kmeans
path_results_base=~/code/dsm010-2021-oct/coursework_01/data/output/british-fiction-corpus-clusters

# the distance metric to use
distance_metrics=("org.apache.mahout.common.distance.CosineDistanceMeasure" "org.apache.mahout.common.distance.EuclideanDistanceMeasure" "org.apache.mahout.common.distance.SquaredEuclideanDistanceMeasure" "org.apache.mahout.common.distance.ManhattanDistanceMeasure")
#distance_metrics=("org.apache.mahout.common.distance.CosineDistanceMeasure")
#distance_metrics=("org.apache.mahout.common.distance.EuclideanDistanceMeasure")

for distance_metric in "${distance_metrics[@]}"
do
  echo "--- Distance Metric: $distance_metric"

    # perform the canopy clustering
    distance_name=${distance_metric##*.}
    path_centroids=dsm010/british-fiction-corpus-canopy-centroids/${distance_name}

    mahout canopy \
      -i $path_vectors \
      -ow \
      -o $path_centroids \
      -dm $distance_metric \
      -t1 0.5 \
      -t2 0.3

  for ((k = $k_start; k <= $k_end; k++))
  do
    # get k with a leading zero if required
    k_padded=$(printf %02d $k)

    # set the output path for the clusters
    path_clusters="${path_clusters_base}/${distance_name}/${k_padded}"

    echo "---- K: $k_padded -- $path_clusters"

    # perform the kmeans clustering
    mahout kmeans \
    -i $path_vectors \
    -c $path_centroids \
    -o "${path_hdfs_base}${path_clusters}" \
    -ow \
    -dm $distance_metric \
    -cl -cd 0.1 -ow -x 20 \
    -k $k

    # set the path for output
    path_final_clusters=`hadoop fs -ls -d -C "${path_clusters}/clusters-*-final"`
    path_clusterpoints="${path_clusters}/clusteredPoints"
    path_results="${path_results_base}/${distance_name}/${k_padded}.txt"

    # output the cluster results
    mahout clusterdump -dt sequencefile \
       -d $path_dictionary \
       -i $path_final_clusters  \
       -o $path_results \
       -of TEXT \
       -b 100 \
       -p $path_clusterpoints \
       -dm $distance_metric \
       -n 20 --evaluate
  done  
done

--- Distance Metric: org.apache.mahout.common.distance.CosineDistanceMeasure
MAHOUT_LOCAL is not set; adding HADOOP_CONF_DIR to classpath.
Running on hadoop, using /opt/hadoop/current/bin/hadoop and HADOOP_CONF_DIR=/opt/hadoop/current/etc/hadoop
MAHOUT-JOB: /opt/mahout/current/mahout-examples-0.13.0-job.jar
---- K: 03 -- dsm010/british-fiction-corpus-kmeans/CosineDistanceMeasure/03
MAHOUT_LOCAL is not set; adding HADOOP_CONF_DIR to classpath.
Running on hadoop, using /opt/hadoop/current/bin/hadoop and HADOOP_CONF_DIR=/opt/hadoop/current/etc/hadoop
MAHOUT-JOB: /opt/mahout/current/mahout-examples-0.13.0-job.jar
MAHOUT_LOCAL is not set; adding HADOOP_CONF_DIR to classpath.
Running on hadoop, using /opt/hadoop/current/bin/hadoop and HADOOP_CONF_DIR=/opt/hadoop/current/etc/hadoop
MAHOUT-JOB: /opt/mahout/current/mahout-examples-0.13.0-job.jar
--- Distance Metric: org.apache.mahout.common.distance.EuclideanDistanceMeasure
MAHOUT_LOCAL is not set; adding HADOOP_CONF_DIR to classpath.
Running o

22/01/01 13:34:05 INFO AbstractJob: Command line arguments: {--distanceMeasure=[org.apache.mahout.common.distance.CosineDistanceMeasure], --endPhase=[2147483647], --input=[dsm010/british-fiction-corpus-vectors/tf-vectors], --method=[mapreduce], --output=[dsm010/british-fiction-corpus-canopy-centroids/CosineDistanceMeasure], --overwrite=null, --startPhase=[0], --t1=[0.5], --t2=[0.3], --tempDir=[temp]}
22/01/01 13:34:05 INFO CanopyDriver: Build Clusters Input: dsm010/british-fiction-corpus-vectors/tf-vectors Out: dsm010/british-fiction-corpus-canopy-centroids/CosineDistanceMeasure Measure: org.apache.mahout.common.distance.CosineDistanceMeasure@4c08a3e0 t1: 0.5 t2: 0.3
22/01/01 13:34:05 INFO DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at lena-master/128.86.245.64:8032
22/01/01 13:34:05 INFO JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/jfoul001/.staging/job_1626049283275_239142
22/01/01 13:34:06 INFO FileInputFormat: Total input fi